In [1]:
from tqdm import tqdm
from utils import dump_jsonl, load_jsonl, load_jsonl_iteratively

import nltk
import spacy
import scispacy
from spacy import displacy

from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector
from scispacy.hyponym_detector import HyponymDetector
from scispacy.custom_tokenizer import combined_rule_tokenizer


jstage_root = "/data/xzhao/experiments/roman-pretrain/datasets/kg-datasets/ja-0.5/eval_qa"
docid_path = f"{jstage_root}/docids.jsonl"
# dataset_path = "/data/xzhao/dataset/roman-pretrain/datasets/medical/en_jstage/data.jsonl"
dataset_path = "/data/xzhao/dataset/roman-pretrain/datasets/medical/ja/data.jsonl"
eval_docids = set(load_jsonl(docid_path))



/home/xzhao/.pyenv/versions/poetry-env/lib/python3.10/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
eval_docs = []
for doc in tqdm(load_jsonl_iteratively(dataset_path)):
    docid = doc["docid"]
    if docid in eval_docids:
        eval_docs.append(doc)
assert len(eval_docs) == len(eval_docids), "Mismatch in number of documents"


614444it [00:05, 114423.36it/s]


In [3]:
print(f"Loading spacy model...")
# spacy.prefer_gpu()

nlp = spacy.load("/data/xzhao/spacy_libs/en_core_sci_scibert/en_core_sci_scibert-0.5.4")
nlp.tokenizer = combined_rule_tokenizer(nlp)

nlp.add_pipe("abbreviation_detector")
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
nlp.add_pipe("hyponym_detector", last=True, config={"extended": True})
linker = nlp.get_pipe("scispacy_linker")


Loading spacy model...


/home/xzhao/.pyenv/versions/poetry-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/xzhao/.pyenv/versions/poetry-env/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/home/xzhao/.pyenv/versions/poetry-env/lib/python3.10/site-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default 

In [4]:
nlp2 = spacy.load("/data/xzhao/spacy_libs/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.5.4")
nlp2.tokenizer = combined_rule_tokenizer(nlp2)

nlp2.add_pipe("abbreviation_detector")
nlp2.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
nlp2.add_pipe("hyponym_detector", last=True, config={"extended": True})
linker2 = nlp2.get_pipe("scispacy_linker")


/home/xzhao/.pyenv/versions/poetry-env/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/xzhao/.pyenv/versions/poetry-env/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
docs = []
for doc in tqdm(eval_docs[:2]):
    data = doc["abstract"]
    sentences = nltk.tokenize.sent_tokenize(data)
    sent_info = []
    for sent in sentences:
        entities = []
        _doc = nlp(sent)
        
        abbreviation = []
        for abrv in _doc._.abbreviations:
            abbreviation.append({
                "text": abrv.text,
                "start": abrv.start_char,
                "end": abrv.end_char,
                "long_form": abrv._.long_form.text
            })
        
        for entity in _doc.ents:
            kg_ents = []
            if entity.label_ == "ORGANISM":
                print(entity.label_, entity.text)
                continue
    
            for umls_ent in entity._.kb_ents:
                kg_ents.append({
                    "cui": umls_ent[0],
                    "score": umls_ent[1],
                    "tui": linker.kb.cui_to_entity[umls_ent[0]].types,
                    "aliases": linker.kb.cui_to_entity[umls_ent[0]].aliases
                })
            assert sent[entity.start_char:entity.end_char] == entity.text, "Mismatch in entity text extraction"
            entities.append({
                "text": entity.text,
                "start": entity.start_char,
                "label": entity.label_,
                "end": entity.end_char,
                "ents": kg_ents
            })

        sent_info.append({
            "text": sent,
            "entities": entities,
            "abbreviations": abbreviation
        })
    doc["sentences"] = sent_info
    docs.append(doc)

  0%|          | 0/2 [00:00<?, ?it/s]/home/xzhao/.pyenv/versions/poetry-env/lib/python3.10/site-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
/home/xzhao/.pyenv/versions/poetry-env/lib/python3.10/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 2/2 [00:01<00:00,  1.42it/s]


In [9]:
doc

{'docid': 'jji1950@@25/2/25_2_101',
 'title': 'Protandrous Hermaphroditism in Six Species of the Anemonefish Genus Amphiprion in Japan',
 'abstract': 'Population structure and gonad histology were investigated in six species of the anemonefish genus Amphiprion occurring in Japanese waters.Species investigated included Amphiprion frenatus, A.clarkii, A.polymnus, A.perideraion, A.sandracinos, and A.ocellaris.All six species occur around their host anemones in small social groups that include a monogamous pair of sexually functional adults and from zero to three sexually non-functional subadults and juveniles.In all species, the single female in a particular social group is larger than her mate, considerably so in some species.Histological examination of gonads indicated that all individuals of all six species are ambosexual as juveniles.At first only testicular tissues mature.Under certain conditions these may later degenerate and the gonads change to ovaries.Field studies supported hist